# A customized observation function

A number of features is collected for each variable that can be branched on.
Features can be divided into differnt classes:

1. **Problem (or graph) features** [STATIC] characterize the problem, e.g., the cycle time, number of workers, lines, workstations, etc.
1. **Variable features** [STATIC] characterize the type of variable, e.g., task-station assignment, worker-station assignment, etc.; the task time if applicable, etc.
1. **Variable features** [DYNAMIC] are calculated features that depend on the partial solution, e.g., the resulting workload if a task is assigned to a station when applicable, etc.

In [1]:
import sys
sys.path.append("..")

import ecole
import pyscipopt
import numpy as np
from sklearn.preprocessing import OneHotEncoder

from models.generator.albp_gen import ALBPGenerator

In [2]:
class ALBPObservation:

    def __init__(self, data) -> None:
        self.data = data

    def before_reset(self, model):
        # disable presolve, heuristics, separation, and propagation
        model.as_pyscipopt().setPresolve(pyscipopt.SCIP_PARAMSETTING.OFF)
        model.as_pyscipopt().setHeuristics(pyscipopt.SCIP_PARAMSETTING.OFF)
        model.as_pyscipopt().setSeparating(pyscipopt.SCIP_PARAMSETTING.OFF)
        model.as_pyscipopt().disablePropagation()
        # collect model information
        m = model.as_pyscipopt()
        nVars = m.getNVars()
        # collect variable types
        types = np.array([str(v)[:1] for v in m.getVars()]).reshape(-1, 1)
        # initialize variable type mask
        varTypeMask = OneHotEncoder().fit(types).transform(types).toarray()
        assert np.all(np.sum(varTypeMask, axis=0))

        # cycle time
        cycleTime = np.ones((nVars, 1)) * self.data['C']

        # task times
        taskTimes = np.zeros((nVars, 1))
        for i, var in enumerate(m.getVars()):
            if "x" in str(var):
                _, j, _ = str(var).split("_")
                taskTimes[i] = self.data['t'][int(j)]

        self.staticFeatures = np.hstack((
            varTypeMask,
            np.ones((nVars, 1)) * self.data['C'],
            taskTimes
        ))

    def extract(self, model, done):
        if done:
            return None
        m = model.as_pyscipopt()
        vars = m.getVars()
        sol = self.read_solution(np.array(vars))

        # the resulting workload
        return self.staticFeatures

    def read_solution(self, vars):
        """
        Read a direct solution representation and return an indirect solution
        representation.
        """
        # ids of branched variables
        b_ids = np.array([i for i in range(len(vars)) if not vars[i].getUbLocal() - vars[i].getLbLocal()], dtype=int)
        print(b_ids)
        branched = vars[b_ids]



In [3]:
instances = ALBPGenerator(
    directory="../data/processed/albp-datasets/SALBP-1993",
    # rng=np.random.default_rng(430)
    rng=np.random.default_rng(430958)
)

instance, data = next(instances)

env = ecole.environment.Branching(
    observation_function=ALBPObservation(data)
)

o, a, r, done, info = env.reset(instance)

original problem has 64 variables (64 bin, 0 int, 0 impl, 0 cont) and 32 constraints
[14 15 16 17 18 19]


In [4]:
for i in np.array(env.model.as_pyscipopt().getVars()):
    print(i)

x_0_1
x_0_2
x_0_3
x_0_4
x_0_5
x_0_6
x_0_7
x_1_1
x_1_2
x_1_3
x_1_4
x_1_5
x_1_6
x_1_7
x_2_1
x_2_2
x_2_3
x_2_4
x_2_5
x_2_6
x_2_7
x_3_1
x_3_2
x_3_3
x_3_4
x_3_5
x_3_6
x_3_7
x_4_1
x_4_2
x_4_3
x_4_4
x_4_5
x_4_6
x_4_7
x_5_1
x_5_2
x_5_3
x_5_4
x_5_5
x_5_6
x_5_7
x_6_1
x_6_2
x_6_3
x_6_4
x_6_5
x_6_6
x_6_7
x_7_1
x_7_2
x_7_3
x_7_4
x_7_5
x_7_6
x_7_7
y_0
y_1
y_2
y_3
y_4
y_5
y_6
y_7


In [5]:
print(done)

False


In [6]:
print(o[a].shape, a.shape)

(4, 4) (4,)


In [7]:
print(o[a])

[[   1.    0. 1000.   11.]
 [   1.    0. 1000.    9.]
 [   1.    0. 1000.    9.]
 [   1.    0. 1000.    5.]]


In [8]:
vars = np.array(env.model.as_pyscipopt().getVars())
# vars[-89]
vars[a]

array([x_0_3, x_2_3, x_2_5, x_3_7], dtype=object)

In [12]:
o

array([[   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   11.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,   17.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    9.],
       [   1.,    0., 1000.,    5.],
       [   1.,    0., 1000.,    5.],
       [   1.,    0., 1000.,    5.],
       [   1.,    0., 1000.,    5.],
       [   1.,    0., 1000.,    5.],
       [   1.,    0., 1000.,    5.],
 